In [1]:
import os
from __future__ import division
import torch 
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable
import numpy as np
import torch.optim as optim
import math
import glob
import glob
from PIL import ImageTk, Image
import numpy as np
import xml.etree.ElementTree as ET
from numpy import savetxt
import gc

from Model.function_bank import *
from Model.Module_layers import *
from Model.Module_layers_backup import *

torch.autograd.set_detect_anomaly(True)
cfgfile = "cfg/yolov4.cfg"
model_file_path = "Model/model.pt"
TL_model_file_path = "Model/TL_model.pt"


In [2]:


anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
mask = [0, 1, 2]
classes = 80
input_image_size = 608

In [3]:
#train only on YOLO layer


"""
Yolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)
Yolo_input = torch.from_numpy(Yolo_input)
#Yolo_run_layer = Yolo(anchors, mask, classes, input_image_size)

#b_x, b_y, b_w, b_h, objective_p, class_p = Yolo_run_layer(Yolo_input)
#combined_yolo_output = torch.cat((b_x, b_y, b_w, b_h, objective_p, class_p), 2)

Yolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)
Yolo_input = torch.from_numpy(Yolo_input)
target = np.array([0 for i in range(3 * 76 * 76 * 85)])
input_tensor = Yolo_input
output_tensor = torch.Tensor(target)

learning_rate = 0.08
epoch_size = 5
steps_for_printing_out_loss = 1

YOLO_Module_WIP = Yolo(anchors, mask, classes, input_image_size)
YOLO_Module_WIP.cuda()
#Model_WIP.to(device)
loss_functioin = nn.MSELoss()
optimizer = optim.SGD(YOLO_Module_WIP.parameters(), lr = learning_rate)

input = input_tensor.cuda()
target = output_tensor.cuda()

def training_model():
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output = YOLO_Module_WIP(input.cuda())
        print(output.size())
        #b_x, b_y, b_w, b_h, objective_p, class_p = YOLO_v4_Module_WIP(input.cuda())
        #output = b_x
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3), output.size(4)))
        loss.backward()
        optimizer.step()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
    torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)

training_model()
"""

"\nYolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)\nYolo_input = torch.from_numpy(Yolo_input)\n#Yolo_run_layer = Yolo(anchors, mask, classes, input_image_size)\n\n#b_x, b_y, b_w, b_h, objective_p, class_p = Yolo_run_layer(Yolo_input)\n#combined_yolo_output = torch.cat((b_x, b_y, b_w, b_h, objective_p, class_p), 2)\n\nYolo_input = np.array([1.0 for i in range(255 * 76 * 76 * 2)]).reshape(2, 255, 76, 76)\nYolo_input = torch.from_numpy(Yolo_input)\ntarget = np.array([0 for i in range(3 * 76 * 76 * 85)])\ninput_tensor = Yolo_input\noutput_tensor = torch.Tensor(target)\n\nlearning_rate = 0.08\nepoch_size = 5\nsteps_for_printing_out_loss = 1\n\nYOLO_Module_WIP = Yolo(anchors, mask, classes, input_image_size)\nYOLO_Module_WIP.cuda()\n#Model_WIP.to(device)\nloss_functioin = nn.MSELoss()\noptimizer = optim.SGD(YOLO_Module_WIP.parameters(), lr = learning_rate)\n\ninput = input_tensor.cuda()\ntarget = output_tensor.cuda()\n\ndef training_model():\n    for i 

In [4]:
#read TOLOv4 model architecture from cfg file
layer_type, layer_details = read_cfg_file(cfgfile)
net_layer = layer_details[0]
layer_type = layer_type[1:]
layer_details = layer_details[1:]

print(len(layer_type))
print(len(layer_details))
print(layer_details[1])

162
162
{'batch_normalize': '1', 'filters': '64', 'size': '3', 'stride': '2', 'pad': '1', 'activation': 'mish'}


In [5]:
# This is used to test end to end structure of YOLOv4
#need to adjust target size according to YOLOv4 output size before testin

input = np.array([1 for i in range(608 * 608 * 3 * 1)]).reshape(1, 3, 608, 608)
#target = np.array([0 for i in range(7 * 7 * 30)])

target = np.array([0 for i in range(3 * 19 * 19 * 85)])

input_tensor = torch.Tensor(input)
output_tensor = torch.Tensor(target)
input = input_tensor.cuda()
target = output_tensor.cuda()

#x = np.array([1 for i in range(608 * 608 * 3)]).reshape(1, 3, 608, 608)
#x = torch.tensor(x)

learning_rate = 0.08
epoch_size = 2
steps_for_printing_out_loss = 1

YOLO_v4_Module_WIP = YOLO_v4_model(layer_details, layer_type)

loss_function = nn.MSELoss()
optimizer = optim.SGD(YOLO_v4_Module_WIP.parameters(), lr = learning_rate)
YOLO_v4_Module_WIP.load_state_dict(torch.load("C:/Users/HX/Desktop/model_YOLOv4.pt")['state_dict'])
YOLO_v4_Module_WIP.eval()
YOLO_v4_Module_WIP.cuda()
"""
for name, param in YOLO_v4_Module_WIP.named_parameters():
    print('name: ', name)
    print(type(param))
    print('param.shape: ', param.shape)
    print('param.requires_grad: ', param.requires_grad)
    print('=====')
#transfer learning:


for name, param in model.named_parameters():
    if name in ['fc.weight', 'fc.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False
"""

def training_model():
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output, b, c = YOLO_v4_Module_WIP(input.cuda())
        #print(output.size())
        #b_x, b_y, b_w, b_h, objective_p, class_p = YOLO_v4_Module_WIP(input.cuda())
        #output = b_x
        #loss = loss_function(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3)))
        global output_tensor
        output_tensor = output
        loss = loss_function(output, target.reshape(output.size(0), output.size(1), output.size(2), output.size(3)))
        loss.backward()
        optimizer.step()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
    #torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)


#training_model()
#torch.save({'output': output_tensor}, 'Model/output.pt')
#YOLO_v4_Module_WIP.state_dict()

In [6]:
m = []

In [7]:
#to use YOLOv4 layer to convert image into layer output from 137, 148, 159

#training_data_image_folder = "F:/FlyAI/UnderwaterDetection_roundA/train-A/image/"
training_data_image_folder = "F:/FlyAI/UnderwaterDetection_roundB/train-B/"
batch_size = 1

image_path_list = glob.glob(training_data_image_folder + "*.jpg")

batch_no = math.ceil(len(image_path_list) / batch_size)
print(batch_no)



#########convert original image into output from YOLOv4
"""
YOLO_v4_Module_WIP = YOLO_v4_model(layer_details, layer_type)
YOLO_v4_Module_WIP.cuda()
YOLO_v4_Module_WIP.load_state_dict(torch.load("C:/Users/HX/Desktop/model_YOLOv4.pt")['state_dict'])
YOLO_v4_Module_WIP.eval()

#for current_batch_no in range(len(image_path_list) // batch_size):
for current_batch_no in range(0, 1200):
    input = image_reader(image_path_list[batch_size * current_batch_no: batch_size * (current_batch_no + 1)])
    input_tensor = torch.Tensor(input).cuda()
    #print(input.shape)
    output_137, output_148, output_159 = YOLO_v4_Module_WIP(input_tensor)
    print(output_137.shape)
    file_name = 'F:/FlyAI/UnderwaterDetection_roundB/TL_input_data/' + str(current_batch_no) + '.pt'
    torch.save({'output_137': output_137, 'output_148': output_148, 'output_159': output_159}, file_name)
    
"""
#torch.save({'output_137': output_137}, 'Model/output.pt')

1200


'\nYOLO_v4_Module_WIP = YOLO_v4_model(layer_details, layer_type)\nYOLO_v4_Module_WIP.cuda()\nYOLO_v4_Module_WIP.load_state_dict(torch.load("C:/Users/HX/Desktop/model_YOLOv4.pt")[\'state_dict\'])\nYOLO_v4_Module_WIP.eval()\n\n#for current_batch_no in range(len(image_path_list) // batch_size):\nfor current_batch_no in range(0, 1200):\n    input = image_reader(image_path_list[batch_size * current_batch_no: batch_size * (current_batch_no + 1)])\n    input_tensor = torch.Tensor(input).cuda()\n    #print(input.shape)\n    output_137, output_148, output_159 = YOLO_v4_Module_WIP(input_tensor)\n    print(output_137.shape)\n    file_name = \'F:/FlyAI/UnderwaterDetection_roundB/TL_input_data/\' + str(current_batch_no) + \'.pt\'\n    torch.save({\'output_137\': output_137, \'output_148\': output_148, \'output_159\': output_159}, file_name)\n    \n'

In [8]:
#all_layerrr


In [9]:
#import training data
#convert image into label data (for training purpose)

round_A_data_folder = "F:/FlyAI/UnderwaterDetection_roundA/"
round_A_image_folder = round_A_data_folder + "train-A/image/"
round_A_box_folder = round_A_data_folder + "train-A/box/"

box_file_path_list = glob.glob(round_A_box_folder + "*.xml")
box_file_name_list = [x.split('\\')[1] for x in box_file_path_list]

image_file_name_list = [(x.split('.')[0] + '.jpg') for x in box_file_name_list]
image_file_path_list = [(round_A_image_folder + x) for x in image_file_name_list]


anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
anchor_shape_1 = int(len(anchors) / 2)
anchors = np.array(anchors).reshape(anchor_shape_1, 2)
def best_anchor(image_info):
    max_IoU = 0
    for m in range(9):
        x_GT = 0
        y_GT = 0
        w_GT = image_info[9]
        h_GT = image_info[10]
        x_PD = image_info[9] / 2 - anchors[m][0] / 2
        y_PD = image_info[10] / 2 - anchors[m][1] / 2
        w_PD = anchors[m][0]
        h_PD = anchors[m][1]
        
        current_IOU = IoU(x_GT, y_GT, w_GT, h_GT, x_PD, y_PD, w_PD, h_PD)
        #print(current_IOU)
        if current_IOU >= max_IoU:
            max_IoU = current_IOU
            selected_anchors = m
    
    if selected_anchors < 3:
        grid_size = 608 / 76
    elif selected_anchors < 6:
        grid_size = 608 / 38
    elif selected_anchors < 9:
        grid_size = 608 / 19
    
    grid_no_x = int((image_info[2] + image_info[4]) / 2 / grid_size)
    grid_no_y = int((image_info[3] + image_info[5]) / 2 / grid_size)
    
    w_expanded_time = image_info[9] / w_PD
    h_expanded_time = image_info[10] / h_PD
    
    position_x = (image_info[2] + image_info[4]) / 2 / grid_size - grid_no_x
    position_y = (image_info[3] + image_info[5]) / 2 / grid_size - grid_no_y
    
    #print(image_info)
    #print(grid_no_x)
    #print(grid_no_y)
    return selected_anchors, grid_no_x, grid_no_y, position_x, position_y, w_expanded_time, h_expanded_time

def read_xml_into_training_data(box_file_path_list):
    All_image = []
    for box_file_path in box_file_path_list:
        dict_object = {"holothurian": 0, "echinus": 1, "scallop": 2, "starfish": 3, 'waterweeds': 0}
        tree = ET.parse(box_file_path)
        root = tree.getroot()
        """
        root.tag
        root.attrib
        
        for child in root:
            for sub_child in child:
                print(sub_child.tag)
        """
        
        object = [None for i in range(6)]
        All_object = []
        All_image_size = []
        image_size = [None, None]
        frame_name = []
        image_size_width = []
        image_size_height = []
        object_name = []
        object_type = []
        object_xmin = []
        object_ymin = []
        object_xmax = []
        object_ymax = []

        for name in root.findall("./frame"):
            frame_name.append(name.text)
        for name in root.findall("./size/width"):
            image_size_width.append(int(name.text))
        for name in root.findall("./size/height"):
            image_size_height.append(int(name.text))

        for name in root.findall("./object/name"):
            object_name.append(name.text)

        for name in root.findall("./object/bndbox/xmin"):
            object_xmin.append(int(name.text))
        for name in root.findall("./object/bndbox/ymin"):
            object_ymin.append(int(name.text))
        for name in root.findall("./object/bndbox/xmax"):
            object_xmax.append(int(name.text))
        for name in root.findall("./object/bndbox/ymax"):
            object_ymax.append(int(name.text))

        for i in range(len(object_name)):
            current_object = []
            current_object.append(object_name[i])
            current_object.append(dict_object[object_name[i]])
            current_object.append(object_xmin[i])
            current_object.append(object_ymin[i])
            current_object.append(object_xmax[i])
            current_object.append(object_ymax[i])
            current_object.append(frame_name[0])
            current_object.append(image_size_width[0])
            current_object.append(image_size_height[0])
            All_object.append(current_object)
        All_image.append(All_object)
    return All_image


training_input_data = read_image(image_file_path_list[0: 2])

All_image = read_xml_into_training_data(box_file_path_list[0:4000])

yolo_size = 608

for i in range(len(All_image)):
    for j in range(len(All_image[i])):
        All_image[i][j][2] = All_image[i][j][2] / All_image[i][j][7] * yolo_size
        All_image[i][j][4] = All_image[i][j][4] / All_image[i][j][7] * yolo_size
        All_image[i][j][3] = All_image[i][j][3] / All_image[i][j][8] * yolo_size
        All_image[i][j][5] = All_image[i][j][5] / All_image[i][j][8] * yolo_size
        All_image[i][j].append(All_image[i][j][4] - All_image[i][j][2])
        All_image[i][j].append(All_image[i][j][5] - All_image[i][j][3])
        selected_anchors, grid_no_x, grid_no_y, position_x, position_y, w_expanded_time, h_expanded_time = best_anchor(All_image[i][j])
        All_image[i][j].append(selected_anchors)
        All_image[i][j].append(grid_no_x)
        All_image[i][j].append(grid_no_y)
        All_image[i][j].append(position_x)
        All_image[i][j].append(position_y)
        All_image[i][j].append(w_expanded_time)
        All_image[i][j].append(h_expanded_time)
        
All_image[0][0]


['holothurian',
 0,
 225.46666666666667,
 233.62962962962962,
 265.3666666666667,
 322.5777777777778,
 '000001',
 1920,
 1080,
 39.900000000000006,
 88.94814814814816,
 3,
 15,
 17,
 0.33854166666666785,
 0.38148148148147953,
 0.08692810457516341,
 0.22181583079338693]

In [12]:
#torch.autograd.set_detect_anomaly(True)
#training loop for transfer layer



#@profile
def TL_model_training():
    learning_rate = 0.001
    epoch_size = 1
    batch_size = 50
    steps_for_printing_out_loss = 1
    
    loss_function_MSE = nn.MSELoss(size_average=False)
    loss_function_BCE = nn.BCELoss(size_average=False)
    
    #loss_function_MSE = nn.MSELoss()
    #loss_function_BCE = nn.BCELoss()
    
    TL_model = transfer_learning_model().cuda()
    optimizer = optim.SGD(TL_model.parameters(), lr = learning_rate)
    #TL_model.load_state_dict(torch.load('Model/TL_model_starting_point.pt')['state_dict'])
    #TL_model.load_state_dict(torch.load('Model/TL_model1.pt')['state_dict'])
    #TL_model.eval()
    
    for i in range(1, epoch_size + 1):
        #loss = 0
        for current_batch_no in range(int(100/ batch_size)):
            optimizer.zero_grad()
            total_loss = 0
            total_loss_w_h = 0
            total_loss_class = 0
            total_loss_obj_p = 0
            total_loss_x_y = 0
            for image_pt_name in range(current_batch_no * batch_size, (current_batch_no + 1) * batch_size):
                torch.cuda.empty_cache()
                if image_pt_name % 200 == 0:
                    print(image_pt_name)

                file_name = 'F:/FlyAI/TL_input_data/' + str(image_pt_name) + '.pt'
                input_data = torch.load(file_name)
                layer_137_out = input_data['output_137'].cuda()
                layer_148_out = input_data['output_148'].cuda()
                layer_159_out = input_data['output_159'].cuda()
                #print(layer_137_out.shape)
                
                output_76, output_38, output_19 = TL_model(layer_137_out, layer_148_out, layer_159_out)
                output_file_name = 'F:/FlyAI/TL_output_data/' + str(image_pt_name) + '.pt'
                torch.save({'output_76': output_76, 'output_38': output_38, 'output_19': output_19}, output_file_name)
                
                target_76 = output_76.clone()
                target_38 = output_38.clone()
                target_19 = output_19.clone()
                #there is a possibility of more than one GT are mapped into same grid of a anchor, may need to check from training data?
                image_set = range(0, 1)
                target_76[:,:,4,:,:] = 0
                target_38[:,:,4,:,:] = 0
                target_19[:,:,4,:,:] = 0
                
                for image_no in image_set:
                    image_no_current_batch = image_no % batch_size

                    for item in All_image[image_pt_name]:
                        #print(item)
                        anchor_no = item[11] % 3
                        grid_x = item[12]
                        grid_y = item[13]

                        obj_class_no = item[1]
                        central_x = (item[2] + item[4]) / 2
                        central_y = (item[3] + item[5]) / 2
                        width_x = item[9]
                        height_y = item[10]

                        position_x = item[14]
                        position_y = item[15]
                        w_expanded_time = item[16]
                        h_expanded_time = item[17]

                        if item[11] < 3:
                            #target_76[anchor_no,image_no_current_batch,4,:,:] = 0
                            target_76[anchor_no,image_no_current_batch,0,grid_y,grid_x] = position_x
                            target_76[anchor_no,image_no_current_batch,1,grid_y,grid_x] = position_y
                            target_76[anchor_no,image_no_current_batch,2,grid_y,grid_x] = w_expanded_time
                            target_76[anchor_no,image_no_current_batch,3,grid_y,grid_x] = h_expanded_time
                            target_76[anchor_no,image_no_current_batch,4,grid_y,grid_x] = 1

                            target_76[anchor_no,image_no_current_batch,5:9,grid_y,grid_x] = 0
                            target_76[anchor_no,image_no_current_batch,5 + obj_class_no,grid_y,grid_x] = 1
                        elif item[11] < 6:
                            #target_38[anchor_no,image_no_current_batch,4,:,:] = 0
                            target_38[anchor_no,image_no_current_batch,0,grid_y,grid_x] = position_x
                            target_38[anchor_no,image_no_current_batch,1,grid_y,grid_x] = position_y
                            target_38[anchor_no,image_no_current_batch,2,grid_y,grid_x] = w_expanded_time
                            target_38[anchor_no,image_no_current_batch,3,grid_y,grid_x] = h_expanded_time
                            target_38[anchor_no,image_no_current_batch,4,grid_y,grid_x] = 1

                            target_38[anchor_no,image_no_current_batch,5:9,grid_y,grid_x] = 0
                            target_38[anchor_no,image_no_current_batch,5 + obj_class_no,grid_y,grid_x] = 1
                        elif item[11] < 9:
                            #target_19[anchor_no,image_no_current_batch,4,:,:] = 0
                            target_19[anchor_no,image_no_current_batch,0,grid_y,grid_x] = position_x
                            target_19[anchor_no,image_no_current_batch,1,grid_y,grid_x] = position_y
                            target_19[anchor_no,image_no_current_batch,2,grid_y,grid_x] = w_expanded_time
                            target_19[anchor_no,image_no_current_batch,3,grid_y,grid_x] = h_expanded_time
                            target_19[anchor_no,image_no_current_batch,4,grid_y,grid_x] = 1

                            target_19[anchor_no,image_no_current_batch,5:9,grid_y,grid_x] = 0
                            target_19[anchor_no,image_no_current_batch,5 + obj_class_no,grid_y,grid_x] = 1

                #global output_tensor
                #output_tensor = output
                #print(output_76.shape)
                #print(output_76)

                target_file_name = 'F:/FlyAI/TL_output_data/target_' + str(image_pt_name) + '.pt'
                torch.save({'target_76': target_76, 'target_38': target_38, 'target_19': target_19}, target_file_name)



                target_76 = Variable(target_76, requires_grad=False)
                target_38 = Variable(target_38, requires_grad=False)
                target_19 = Variable(target_19, requires_grad=False)

                #print(target_76[0])

                loss_w_h = loss_function_MSE(output_76[:, :, 2 : 4, :, :], target_76[:, :, 2 : 4, :, :])\
                 + loss_function_MSE(output_38[:, :, 2 : 4, :, :], target_38[:, :, 2 : 4, :, :])\
                 + loss_function_MSE(output_19[:, :, 2 : 4, :, :], target_19[:, :, 2 : 4, :, :])

                loss_w_h = loss_w_h / 2

                loss_class = loss_function_BCE(output_76[:, :, 5 : 9, :, :], target_76[:, :, 5 : 9, :, :])\
                 + loss_function_BCE(output_38[:, :, 5 : 9, :, :], target_38[:, :, 5 : 9, :, :])\
                 + loss_function_BCE(output_19[:, :, 5 : 9, :, :], target_19[:, :, 5 : 9, :, :])

                loss_obj_p = loss_function_BCE(output_76[:, :, 4, :, :], target_76[:, :, 4, :, :])\
                 + loss_function_BCE(output_38[:, :, 4, :, :], target_38[:, :, 4, :, :])\
                 + loss_function_BCE(output_19[:, :, 4, :, :], target_19[:, :, 4, :, :])

                loss_x_y = loss_function_BCE(output_76[:, :, 0 : 2, :, :], target_76[:, :, 0 : 2, :, :])\
                 + loss_function_BCE(output_38[:, :, 0 : 2, :, :], target_38[:, :, 0 : 2, :, :])\
                 + loss_function_BCE(output_19[:, :, 0 : 2, :, :], target_19[:, :, 0 : 2, :, :])

                loss = loss_w_h + loss_class + loss_obj_p + loss_x_y
                #loss = loss_function_MSE(output_76, target_76) + loss_function_MSE(output_38, target_38) + loss_function_MSE(output_19, target_19)

                total_loss += loss
                total_loss_w_h += loss_w_h
                total_loss_class += loss_class
                total_loss_obj_p += loss_obj_p
                total_loss_x_y += loss_x_y
                gc.collect()
                print('Max memory allocated: {0:.2f} MB'
                      .format(torch.cuda.max_memory_allocated() / 1e6))
                print('Max memory cached: {0:.2f} MB'
                      .format(torch.cuda.max_memory_cached() / 1e6))
                loss.backward()

            #loss = loss_function(output_76, target_76)

            optimizer.step()
            if i % (steps_for_printing_out_loss) == 0:
                print('Loss (epoch: ' + str(i) + '): ' + str(total_loss.cpu().detach().numpy()))
                print(total_loss_w_h)
                print(total_loss_class)
                print(total_loss_obj_p)
                print(total_loss_x_y)
                TL_model_file_path_by_epoch = "Model/TL_model"+ str(i) + ".pt" 
                torch.save({'state_dict': TL_model.state_dict(),'optimizer': optimizer.state_dict()}, TL_model_file_path_by_epoch)
            
    torch.save({'state_dict': TL_model.state_dict(),'optimizer': optimizer.state_dict()}, TL_model_file_path)

TL_model_training()

D:\Software\anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


0
Max memory allocated: 284.12 MB
Max memory cached: 291.50 MB
Max memory allocated: 308.58 MB
Max memory cached: 316.67 MB
Max memory allocated: 329.55 MB
Max memory cached: 339.74 MB
Max memory allocated: 350.52 MB
Max memory cached: 360.71 MB
Max memory allocated: 371.49 MB
Max memory cached: 381.68 MB
Max memory allocated: 392.46 MB
Max memory cached: 402.65 MB
Max memory allocated: 413.43 MB
Max memory cached: 423.62 MB
Max memory allocated: 434.41 MB
Max memory cached: 444.60 MB
Max memory allocated: 455.38 MB
Max memory cached: 465.57 MB
Max memory allocated: 476.35 MB
Max memory cached: 486.54 MB
Max memory allocated: 497.32 MB
Max memory cached: 507.51 MB
Max memory allocated: 518.29 MB
Max memory cached: 528.48 MB
Max memory allocated: 539.26 MB
Max memory cached: 549.45 MB
Max memory allocated: 560.24 MB
Max memory cached: 570.43 MB
Max memory allocated: 581.21 MB
Max memory cached: 591.40 MB
Max memory allocated: 602.18 MB
Max memory cached: 612.37 MB
Max memory allocated: 

In [ ]:
#print(image_pt_name)

In [ ]:
def target_into_csv(image_pt_name):
    target_file_name = 'F:/FlyAI/TL_output_data/target_' + str(image_pt_name) + '.pt'

    target_data = torch.load(target_file_name)
    target_76 = target_data['target_76']
    target_38 = target_data['target_38']
    target_19 = target_data['target_19']

    target_76 = torch.transpose(target_76, 2, 4)
    target_38 = torch.transpose(target_38, 2, 4)
    target_19 = torch.transpose(target_19, 2, 4)

    target_76 = target_76.reshape(3 * 76**2, 9)
    target_38 = target_38.reshape(3 * 38**2, 9)
    target_19 = target_19.reshape(3 * 19**2, 9)

    #print(target_38[:,:,5,:,:])
    savetxt('19_data_t.csv', target_19.cpu().detach().numpy(), delimiter=',')
    savetxt('38_data_t.csv', target_38.cpu().detach().numpy(), delimiter=',')
    savetxt('76_data_t.csv', target_76.cpu().detach().numpy(), delimiter=',')
    
target_into_csv(10)

In [ ]:
def training_input_into_csv(image_pt_name):
    input_file_name = 'F:/FlyAI/TL_input_data/' + str(image_pt_name) + '.pt'

    input_data = torch.load(input_file_name)
    input_76 = input_data['output_137'].cuda()
    input_38 = input_data['output_148'].cuda()
    input_19 = input_data['output_159'].cuda()

    input_76 = torch.transpose(input_76, 1, 3)
    input_38 = torch.transpose(input_38, 1, 3)
    input_19 = torch.transpose(input_19, 1, 3)

    input_76 = input_76.reshape(76**2, 256)
    input_38 = input_38.reshape(38**2, 512)
    input_19 = input_19.reshape(19**2, 1024)

    savetxt('76_data_i.csv', input_76.cpu().detach().numpy(), delimiter=',')
    savetxt('38_data_i.csv', input_38.cpu().detach().numpy(), delimiter=',')
    savetxt('19_data_i.csv', input_19.cpu().detach().numpy(), delimiter=',')
    
training_input_into_csv(10)

In [ ]:
def training_output_into_csv(image_pt_name):
    output_file_name = 'F:/FlyAI/TL_output_data/' + str(image_pt_name) + '.pt'

    output_data = torch.load(output_file_name)
    output_76 = output_data['output_76'].cuda()
    output_38 = output_data['output_38'].cuda()
    output_19 = output_data['output_19'].cuda()

    output_76 = torch.transpose(output_76, 2, 4)
    output_38 = torch.transpose(output_38, 2, 4)
    output_19 = torch.transpose(output_19, 2, 4)

    output_76 = output_76.reshape(3 * 76**2, 9)
    output_38 = output_38.reshape(3 * 38**2, 9)
    output_19 = output_19.reshape(3 * 19**2, 9)

    savetxt('19_data_o.csv', output_19.cpu().detach().numpy(), delimiter=',')
    savetxt('38_data_o.csv', output_38.cpu().detach().numpy(), delimiter=',')
    savetxt('76_data_o.csv', output_76.cpu().detach().numpy(), delimiter=',')

training_output_into_csv(10)

In [ ]:
sum(x**2 for x in (19, 38, 76))
def x():
    output = torch.cat((output_76, output_38, output_19), 0)
    print(output.shape)
    obj_threshold = 0.53
    updated_outcome = []
    for item in output:
        if item[4] >= obj_threshold:
            updated_outcome.append(item)

In [ ]:
#len(updated_outcome)

In [ ]:


#len(updated_outcome)
#updated_outcome = updated_outcome[0:9]
#print(updated_outcome.shape)
def sort_column(elem):
    return elem[4]
#updated_outcome.sort(key = sort_column, reverse=True)

#updated_outcome = updated_outcome[updated_outcome[:,0].argsort()]

#print(updated_outcome)

def x():
    #sort by obj possibility
    IoU_threshold = 0.5
    final_list = []
    #print(updated_outcome)
    #updated_outcome[0][3] = 6

    for origin_item in updated_outcome:
        #print(final_list)
        if final_list == []:
            final_list.append(origin_item)
            continue
        indicator = 1
        for new_item in final_list:
            #print(IoU(new_item[0] - new_item[2]/2, new_item[1] - new_item[3]/2, new_item[2], new_item[3], origin_item[0] - origin_item[2]/2, origin_item[1] - origin_item[3]/2, origin_item[2], origin_item[3]))
            if IoU(new_item[0] - new_item[2]/2, new_item[1] - new_item[3]/2, new_item[2], new_item[3], origin_item[0] - origin_item[2]/2, origin_item[1] - origin_item[3]/2, origin_item[2], origin_item[3]) < IoU_threshold:
                indicator *= 1
            else:
                indicator *= 0
                break
        if indicator == 1:
            final_list.append(origin_item)

    final_list


In [ ]:

class transfer_learning_model_prediction(nn.Module):
    def __init__(self):
        super().__init__()
        self.anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
        self.mask_a = [0, 1, 2]
        self.mask_b = [3, 4, 5]
        self.mask_c = [6, 7, 8]
        #self.TL_model_list = nn.ModuleList()
        #TL_model_list.append(Conv_Layer_box(in_channel = 256, out_channel = 255, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalize = False))
        self.Conv_Layer_76_a = Conv_Layer_box(in_channel = 256, out_channel = 255, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #
        self.norm_76 = nn.BatchNorm2d(255)
        self.Sigmoid_layer_76 = nn.Sigmoid()
        self.Conv_Layer_76_b = Conv_Layer_box(in_channel = 255, out_channel = 27, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #self.Yolo_Layer_76 = Yolo(self.anchors, self.mask_a, classes = 4, input_image_size = 608)
        self.Yolo_Layer_76 = Yolo_TL_prediction(self.anchors, self.mask_a, classes = 4, input_image_size = 608)
        
        self.Conv_Layer_38_a = Conv_Layer_box(in_channel = 512, out_channel = 255, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #
        self.norm_38 = nn.BatchNorm2d(255)
        self.Sigmoid_layer_38 = nn.Sigmoid()
        self.Conv_Layer_38_b = Conv_Layer_box(in_channel = 255, out_channel = 27, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #self.Yolo_Layer_38 = Yolo(self.anchors, self.mask_b, classes = 4, input_image_size = 608)
        self.Yolo_Layer_38 = Yolo_TL_prediction(self.anchors, self.mask_b, classes = 4, input_image_size = 608)
        
        self.Conv_Layer_19_a = Conv_Layer_box(in_channel = 1024, out_channel = 255, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #
        self.norm_19 = nn.BatchNorm2d(255)
        self.Sigmoid_layer_19 = nn.Sigmoid()
        self.Conv_Layer_19_b = Conv_Layer_box(in_channel = 255, out_channel = 27, kernel_size = 1, stride = 1, activation_func = 'linear', batch_normalization = False)
        #self.Yolo_Layer_19 = Yolo(self.anchors, self.mask_c, classes = 4, input_image_size = 608)
        self.Yolo_Layer_19 = Yolo_TL_prediction(self.anchors, self.mask_c, classes = 4, input_image_size = 608)
        
    def forward(self, layer_137_out, layer_148_out, layer_159_out, image_no):
        out_76_1 = self.Conv_Layer_76_a(layer_137_out)
        
        #out_76_1 = self.norm_76(out_76_1)
        out_76_1 = self.Sigmoid_layer_76(out_76_1)
        out_76_1 = self.Conv_Layer_76_b(out_76_1)
        out_76_1 = self.Yolo_Layer_76(out_76_1)
        
        out_38_1 = self.Conv_Layer_38_a(layer_148_out)
        
        #out_38_1 = self.norm_76(out_38_1)
        out_38_1 = self.Sigmoid_layer_38(out_38_1)
        out_38_1 = self.Conv_Layer_38_b(out_38_1)
        out_38_1 = self.Yolo_Layer_38(out_38_1)
        
        out_19_1 = self.Conv_Layer_19_a(layer_159_out)
        
        #out_19_1 = self.norm_76(out_19_1)
        out_19_1 = self.Sigmoid_layer_19(out_19_1)
        out_19_1 = self.Conv_Layer_19_b(out_19_1)
        out_19_1 = self.Yolo_Layer_19(out_19_1)
        #another option: only use 1 conv layer b, and Yolo layer
        
        output_76 = torch.transpose(out_76_1, 2, 4)
        output_38 = torch.transpose(out_38_1, 2, 4)
        output_19 = torch.transpose(out_19_1, 2, 4)

        output_76 = output_76.reshape(3 * 76**2, 9)
        output_38 = output_38.reshape(3 * 38**2, 9)
        output_19 = output_19.reshape(3 * 19**2, 9)
        
        output = torch.cat((output_76, output_38, output_19), 0)
        print(output.shape)
        #print(output)
        #savetxt('abc.csv', output.cpu().detach().numpy(), delimiter=',')
        def sort_column(elem):
            return elem[4]
        output = output.tolist()
        output.sort(key = sort_column, reverse=True)
        
        obj_threshold = 0.8
        updated_outcome = []
        for item in output:
            if item[4] >= obj_threshold:
                updated_outcome.append(item)
            else:
                break
        IoU_threshold = 0.7
        final_list = []
        #print(updated_outcome)
        #updated_outcome[0][3] = 6
        FlyAI_result = []
        for origin_item in updated_outcome:
            #print(final_list)
            if final_list == []:
                final_list.append(origin_item)
                continue
            indicator = 1
            for new_item in final_list:
                #print(IoU(new_item[0] - new_item[2]/2, new_item[1] - new_item[3]/2, new_item[2], new_item[3], origin_item[0] - origin_item[2]/2, origin_item[1] - origin_item[3]/2, origin_item[2], origin_item[3]))
                if IoU(new_item[0] - new_item[2]/2, new_item[1] - new_item[3]/2, new_item[2], new_item[3], origin_item[0] - origin_item[2]/2, origin_item[1] - origin_item[3]/2, origin_item[2], origin_item[3]) < IoU_threshold:
                    indicator *= 1
                else:
                    indicator *= 0
                    break
            if indicator == 1:
                final_list.append(origin_item)
        self.mrr = []
        self.result = []
        for item in final_list:
            a = new_item[5:9]
            name = a.index(max(a))
            image_id = image_no
            confidence = new_item[4]
            xmin = round(new_item[0] - new_item[2]/2)
            ymin = round(new_item[1] - new_item[3]/2)
            xmax = round(new_item[0] + new_item[2]/2)
            ymax = round(new_item[1] + new_item[3]/2)
            
            self.mrr = [name, image_id, confidence, xmin, ymin, xmax, ymax]
            self.result.append(self.mrr)
        return self.result

In [ ]:

TL_model_prediction = transfer_learning_model_prediction().cuda()

TL_model_prediction.load_state_dict(torch.load('Model/TL_model2.pt')['state_dict'])
TL_model_prediction.eval()

submission_list = []
for image_pt_name in range(1):
    torch.cuda.empty_cache()
    file_name = 'F:/FlyAI/UnderwaterDetection_roundB/TL_input_data/' + str(image_pt_name) + '.pt'
    input_data = torch.load(file_name)
    layer_137_out = input_data['output_137'].cuda()
    layer_148_out = input_data['output_148'].cuda()
    layer_159_out = input_data['output_159'].cuda()
    kkkk = TL_model_prediction(layer_137_out, layer_148_out, layer_159_out, str(image_pt_name))
    submission_list.append(kkkk)

In [ ]:
submission_list

In [ ]:
len(final_list)

In [ ]:
3 * (19**2) * (1 + 4 + 16)

In [ ]:
#loss function - Done

"""
- objective possibility: except GT grids are 1, all others are 0
- class: 0, 1 only for GT grids, rest should be ignore (equal to predicted result)
- x, y, w, h only for GT grids, rest should be ignore (equal to predicted result)


"""
#grab yolov4 weight into TL - Done
#convert training data into 60GB mid input data
##########NMS (possibility threshold, IOU threshold, max 80)

In [ ]:
#block 1 layer's parameter



#convert result into FlyAI format
#change back to normal pic size

In [ ]:
layer_details[160]

In [ ]:
#all_layers
n = [76, 38, 19]
m = [608 / i for i in n for m in range(6)] 
print(m)


anchor = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
[anchor[i] / m[i] for i in range(len(m))]

In [ ]:
class YOLO_v4_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.YOLO_v4_layers = nn.ModuleList()
        
            self.YOLO_v4_layers.append(Conv_Layer_box(in_channel[i], out_channel[i], kernel_size= kernel_size[i], stride = stride[i], activation_func = activation_func[i], batch_normalization = batch_normalization[i]))
    

In [ ]:
class abc():
    def __init__(self, qwe, out):
        print(qwe)
        
abc(4,5)

In [ ]:
abc = "1,2,3"
m = abc.split(",")
m
abc = "1"
m = abc.split(",")
m
k = 4

HX = []

for r in k:
    print(r)
    print(k[r])

In [ ]:
weightfile = "D:/Installation/yolov4.weights"
fp = open(weightfile, 'rb')
header = np.fromfile(fp, count=5, dtype=np.int32)
header = torch.from_numpy(header)
seen = header[3]
buf = np.fromfile(fp, dtype=np.float32)
fp.close()

start = 0
ind = -2
buf


In [ ]:
HX_weight = YOLO_v4_Module_WIP.state_dict()
i = 0
HX = []

for kk in HX_weight:
    i += 1
    print(kk)
    print(HX_weight[kk].size())
    HX.append(HX_weight[kk].size())
print(i)

In [ ]:
d = torch.load("D:/Installation/yolov4.pt")
#d

In [ ]:
mm = d['model']
weight_bank = []
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk].size())
    weight_bank.append(mm[kk])
    yolo_v4_size.append(mm[kk].size())
print(weight_bank)
print(i)

In [ ]:
weight_bank[0]

In [ ]:
len(weight_bank)
#update weight into model

i = 0
HX = []

kk_list = []
for kk in YOLO_v4_Module_WIP.state_dict():
    kk_list.append(kk)
#print(kk_list)

for kk in kk_list:
    #print(weight_bank[i])
    #print(kk)
    #print(YOLO_v4_Module_WIP.state_dict()[kk])
    YOLO_v4_Module_WIP['state_dict'].update({kk:  weight_bank[i]})
    
    #print("------------------------------")
    print(YOLO_v4_Module_WIP['state_dict'][kk])
    i += 1
    #HX.append(HX_weight[kk].size())
print(i)


In [ ]:
#YOLO_v4_Module_WIP.state_dict()[kk_list[0]]
#YOLO_v4_Module_WIP['state_dict']

YOLOv4_wieght = torch.load("C:/Users/HX/Desktop/model.pt")
i = 0
fff = YOLOv4_wieght['state_dict']

for kk in kk_list:
    #print(weight_bank[i])
    #print(kk)
    #print(YOLO_v4_Module_WIP.state_dict()[kk])
    fff.update({kk:  weight_bank[i]})
    i += 1
    #print("------------------------------")
print(fff)

In [ ]:
torch.save({'state_dict': fff,'optimizer': optimizer.state_dict()}, "C:/Users/HX/Desktop/model_YOLOv4.pt")

In [ ]:
for i in range(len(yolo_v4_size)):
    #print(yolo_v4_size[i] == HX[i])
    if (yolo_v4_size[i] == HX[i]) == False:
        print(yolo_v4_size[i])
        print(HX[i])
        print(i)
    print(yolo_v4_size[i] == HX[i])



In [ ]:
#YOLO_v4_Module_WIP.state_dict()
torch.save({'state_dict': YOLO_v4_Module_WIP.state_dict(),'optimizer': optimizer.state_dict()}, model_file_path)

In [ ]:
#TL parameter:
d = torch.load("D:/Installation/yolov4.pt")
mm = d['model']
weight_bank = []
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk])
    weight_bank.append(mm[kk])
    yolo_v4_size.append(mm[kk].size())
print(i)



In [ ]:
#TL parameter:
d = torch.load("C:/Users/HX/Desktop/model.pt")
mm = d['state_dict']
weight_bank = []
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk])
    weight_bank.append(mm[kk])
    yolo_v4_size.append(mm[kk].size())
print(i)

In [ ]:
#TL parameter:
d = torch.load(TL_model_file_path)
mm = d['state_dict']
weight_bank = []
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk].size())
    weight_bank.append(mm[kk])
    yolo_v4_size.append(mm[kk].size())
print(i)




In [ ]:
#len(weight_bank)
#update weight into model
TL_model_weight = torch.load(TL_model_file_path)
YOLO_v4_weight = torch.load("D:/Installation/yolov4.pt")['model']
i = 0
HX = []

TL_model_weight = TL_model_weight['state_dict']

#print(TL_model_weight['state_dict']['Conv_Layer_76_a.conv_box.0.weight'])

TL_model_weight.update({'Conv_Layer_76_a.conv_box.0.weight': YOLO_v4_weight['module_list.138.Conv2d.weight']})
TL_model_weight.update({'Conv_Layer_76_a.conv_box.0.bias': YOLO_v4_weight['module_list.138.Conv2d.bias']})
TL_model_weight.update({'Conv_Layer_38_a.conv_box.0.weight': YOLO_v4_weight['module_list.149.Conv2d.weight']})
TL_model_weight.update({'Conv_Layer_38_a.conv_box.0.bias': YOLO_v4_weight['module_list.149.Conv2d.bias']})
TL_model_weight.update({'Conv_Layer_19_a.conv_box.0.weight': YOLO_v4_weight['module_list.160.Conv2d.weight']})
TL_model_weight.update({'Conv_Layer_19_a.conv_box.0.bias': YOLO_v4_weight['module_list.160.Conv2d.bias']})

print(TL_model_weight['Conv_Layer_76_a.conv_box.0.weight'])

torch.save({'state_dict': TL_model_weight}, 'Model/TL_model_starting_point.pt')
#YOLO_v4_weight['module_list.138.Conv2d.weight']

In [ ]:
mm = TL_model_weight['state_dict']
weight_bank = []
i = 0
yolo_v4_size = []
for kk in mm:
    i += 1
    print(kk)
    print(mm[kk].size())
    weight_bank.append(mm[kk])
    yolo_v4_size.append(mm[kk].size())
print(i)


In [ ]:
for i in range(4):
    if i == 2:
        continue
    print(i)